# Trabalho Prático 1
## Algoritmos Geométricos
**Proposta:** Envoltória Convexa como algoritmo de classificação  
**Desenvolvedor:** Pedro de Oliveira Guedes

# Introdução
Falar sobre a motivação do trabalho, o que deve ser feito e como será testado/avaliado ao final

# Implementação
Descrever como a implementação dos algoritmos relativos ao trabalho será feita ao longo do documento.  
A cada subtítulo desta seção, deverá existir uma explicação da escolha daquele algoritmo, bem como o que e como ele faz e uma análise da complexidade do mesmo.

## Bibliotecas utilizadas
As bibliotecas utilizadas na implementação dos algoritmos deste trabalho tem como função facilitar o desenvolvimento, utilizando algoritmos conhecidos de Algebra Linear, como redução PCA, SVD, cálculo de determinantes, entre outros.

In [2]:
import numpy as np

## Envoltória convexa
O algoritmo escolhido para realizar esta tarefa é o da **Varredura de Graham**, que possui complexidade de tempo O(n*log n) e de espaço O(n). Ele foi escolhido em detrimento do algoritmo **Gift Wrapping** proposto por Jarvis, devido à menor complexidade de tempo no pior caso. Como a distribuição dos pontos após a redução de dimensionalidade dos dados é imprevisível, não é possível afirmar para todos os casos que a quantidade de vértices será mínima.
  
Esse algoritmo funciona fazendo a ordenação dos pontos no espaço bidimensional em relação a um vértice denominado âncora, que é escolhido como sendo o que possui menor coordenada Y, utilizando a menor coordenada X como critério de desempate.  
  
Após o encontro do âncora e a ordenação dos pontos, o algoritmo itera por cada um dos vértices verificando se eles devem ou não ser incluídos como vértices da envoltória, ou se eles já fazem parte como ponto interno da mesma. Para verificar se um ponto *p3* será ou não incluído como vértice do polígono, é averiguada a direção do giro feito pelo segmento *p2 -> p3* em relação a *p1 -> p2*.  
  
Caso o giro seja nulo ou à esquerda, o ponto p3 é incluído como vértice da envoltória. Caso contrário, o ponto *p2* é descartado e a mesma verificação é feita para o segmento *p1 -> p3* em relação a *p0 -> p1*.

### Critério de comparação
Para realizar a comparação entre dois pontos distintos, é necessário realizar a normalização de ambos em relação ao âncora do conjunto. Sendo assim, será definida uma nova classe na qual os pontos serão descritos, que armazena o âncora e sobrescreve métodos de comparação como *<* e *>*, o que facilita a ordenação dos pontos.

In [12]:
def turns_left_or_keeps_direction(p0: tuple, p1: tuple, p2: tuple): # Similar a <=
    matrix = np.array([
            [(p1[0] - p0[0]), (p1[1] - p0[1])],
            [(p2[0] - p0[0]), (p2[1] - p0[1])]
        ])
    return np.linalg.det(matrix) <= 0.0

class Vertix:
    def __init__(self, anchor: tuple, point: tuple):
        self.anchor = anchor
        self.point = point

    def __repr__(self):
        return f'({self.point[0]}, {self.point[1]})'

    def __lt__(self, other):
        return turns_left_or_keeps_direction(p0 = self.anchor, p1 = self.point, p2 = other.point)

### Encontrando o âncora
Para encontrar o vértice âncora do conjunto, o algoritmo assumirá que o primeiro ponto é o âncora inicialmente. Após isso, será iterado pelos outros vértices do conjunto, verificando se o atual âncora está acima do atual vértice comparado.  
  
Caso ele esteja, o atual vértice vira o novo âncora. Caso ambos estejam em uma mesma coordenada Y, então o que possuir a menor coordenada X será considerado âncora.

In [13]:
def find_anchor(point_set: list[tuple]) -> tuple:
    current_anchor = point_set[0]
    
    for index in range(1, len(point_set)):
        if point_set[index][1] < current_anchor[1]: # Comparando Y
            current_anchor = point_set[index]
        if point_set[index][1] == current_anchor[1]:
            current_anchor = point_set[index] if point_set[index][0] < current_anchor[0] else current_anchor # Comparando X no caso de empate

    return current_anchor

### Ordenando os segmentos
A ordenação dos segmentos será baseada no critério de comparação definido anteriormente.  

Para isso, o âncora será removido do conjunto inicialmente e readicionado ao fim. Para que a ordenação ocorra de forma facilitada, os elementos do conjunto serão transformados no tipo **Vertix** definido acima.

In [14]:
def sort_points_set(anchor: tuple, points_set: list[tuple]) -> list[tuple]:
    points_set.remove(anchor)

    vertix_set = [Vertix(anchor, (point[0], point[1])) for point in points_set]

    vertix_set.sort()

    sorted_points = [vertix.point for vertix in vertix_set]

    return sorted_points.insert(0, anchor)

### Algoritmo final
Após as definições anteriores, o algoritmo final pode ser montado de acordo com o funcionamento que foi descrito no texto da seção **Envoltória Convexa**.

In [23]:
def graham_scan(points_set: list[tuple]) -> list[tuple]:
    anchor = find_anchor(point_set = points_set)
    
    sorted_points = sort_points_set(anchor = anchor, points_set = points_set)

    convex_hull_vertices = [point for point in sorted_points[:3]]

    for current_point in sorted_points[3:]:
        top = convex_hull_vertices.pop()
        next_to_top = convex_hull_vertices.pop()
        
        while not turns_left_or_keeps_direction(p0 = next_to_top, p1 = top, p2 = current_point):
            top = next_to_top
            next_to_top = convex_hull_vertices.pop()

        convex_hull_vertices.append(next_to_top)
        convex_hull_vertices.append(top)
        convex_hull_vertices.append(current_point)

    return convex_hull_vertices

## Verificação de separabilidade linear
Blá blá blá

### Detecção de interseções em conjuntos de segmentos
Blá blá blá

### Definição da separabilidade linear
Blá blá blá

## Construção do modelo
Blá blá blá

## Classificador
Blá blá blá

## Representação gráfica da classificação
Blá blá blá

## Metrificação do classificador
Blá blá blá

# Testes do algoritmo
Dizer de onde estão surgindo as fontes de dados, como elas são tratadas e os desafios relacionados a uma classificação eficiente no algoritmo implementado

## Redutor de dimensionalidade
Falar qual foi a estratégia empregada para a redução e como ela deve ser utilizada.

## Teste 1 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 2 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 3 (2D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade não necessita de tratamento e que isso pode levar a melhores resultados de classificação.

## Teste 4 (3D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 5 (3D)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 6 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 7 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 8 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 9 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

## Teste 10 (XD)
Dizer como a base foi obtida, comentar o fato de que a dimensionalidade necessita de tratamento e que isso pode levar a piores resultados de classificação.

# Conclusão
Comentar sobre como o experimento contribuiu para o aprendizado da matéria, como o resultado dos testes foi diferente do ideal, etc...